In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46728
2,Huelva,Confirmados PDIA 14 días,521
3,Huelva,Tasa PDIA 14 días,"101,5258101603757"
4,Huelva,Confirmados PDIA 7 días,272
5,Huelva,Tasa PDIA 7 dias,"53,00387785724029"
6,Huelva,Total Confirmados,46931
7,Huelva,Curados,45501
8,Huelva,Fallecidos,421


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46728.0


/tmp/ipykernel_66943/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12870.0


/tmp/ipykernel_66943/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_66943/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_66943/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_66943/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1498 personas en los últimos 7 días 

Un positivo PCR cada 846 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46728.0,272.0,521.0,513170.0,53.003878,101.525810,29.0
Huelva-Costa,27623.0,154.0,289.0,289548.0,53.186346,99.810739,22.0
Huelva (capital),12870.0,96.0,170.0,143837.0,66.742215,118.189339,16.0
Condado-Campiña,14547.0,100.0,193.0,156231.0,64.007783,123.535022,7.0
Cartaya,2190.0,33.0,63.0,20083.0,164.318080,313.698153,5.0
Palma del Condado (La),1450.0,10.0,15.0,10801.0,92.584020,138.876030,2.0
Almonte,2355.0,10.0,26.0,24507.0,40.804668,106.092137,1.0
Ayamonte,2115.0,2.0,4.0,21104.0,9.476876,18.953753,1.0
Moguer,2060.0,19.0,34.0,21867.0,86.888919,155.485435,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,59.0,1.0,7.0,713.0,140.252454,981.767181,0.0
Palos de la Frontera,1123.0,25.0,52.0,11742.0,212.910918,442.854710,1.0
Granada de Río-Tinto (La),6.0,1.0,1.0,243.0,411.522634,411.522634,0.0
Almonaster la Real,92.0,1.0,6.0,1788.0,55.928412,335.570470,0.0
Cartaya,2190.0,33.0,63.0,20083.0,164.318080,313.698153,5.0
San Bartolomé de la Torre,394.0,6.0,8.0,3761.0,159.532039,212.709386,1.0
Bollullos Par del Condado,1326.0,14.0,28.0,14387.0,97.310072,194.620143,0.0
Moguer,2060.0,19.0,34.0,21867.0,86.888919,155.485435,1.0
Fuenteheridos,14.0,1.0,1.0,709.0,141.043724,141.043724,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Campofrío,59.0,1.0,7.0,713.0,140.252454,981.767181,0.0,0.142857
Almonaster la Real,92.0,1.0,6.0,1788.0,55.928412,335.570470,0.0,0.166667
Lepe,3056.0,3.0,9.0,27880.0,10.760402,32.281205,0.0,0.333333
Almonte,2355.0,10.0,26.0,24507.0,40.804668,106.092137,1.0,0.384615
Aljaraque,1653.0,4.0,10.0,21474.0,18.627177,46.567943,0.0,0.400000
Sierra de Huelva-Andévalo Central,4130.0,16.0,37.0,67391.0,23.742043,54.903474,0.0,0.432432
Isla Cristina,3189.0,12.0,27.0,21393.0,56.093115,126.209508,0.0,0.444444
Palos de la Frontera,1123.0,25.0,52.0,11742.0,212.910918,442.854710,1.0,0.480769
Rociana del Condado,676.0,1.0,2.0,7939.0,12.596045,25.192090,0.0,0.500000
